In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

import matplotlib.pyplot as plt

from sklearn.svm import LinearSVR
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [2]:
df_101 = pd.read_excel('./Datasets/Raw Data_Room 101.xlsx')

# 清理缺省值

In [4]:
def clean_NaN(df_Room,columns_Time,columns_Name):
    df_Room_Column =  pd.DataFrame(df_Room,columns = [columns_Time,columns_Name]).dropna(axis=0,how='any').reset_index().drop(['index'],axis=1)
    return df_Room_Column

In [5]:
df_101_Alarm = clean_NaN(df_101,'Time_0','AlarmSignal_0101')
df_101_State = clean_NaN(df_101,'Time_1','OnOffState_0101')
df_101_Mode = clean_NaN(df_101,'Time_2','OperationModeState_0101')
df_101_InTemp = clean_NaN(df_101,'Time_3','RoomTemp_0101')
df_101_SetTemp = clean_NaN(df_101,'Time_4','SetTemp_0101')
df_101_OutHum = clean_NaN(df_101,'Time_5','OUTAIRHUMD')
df_101_OutTemp = clean_NaN(df_101,'Time_6','OUTAIRTEMP')

# 清理重复值并修改日期形式

In [6]:
def transFormat_CleanDuplicated(df,column_names):
    df[column_names]=pd.to_datetime(df[column_names], format='%d.%m.%Y %H:%M:%S')
    df = df.drop_duplicates(column_names,keep ='first')
    df_Res = df.set_index(column_names)
    return df_Res

df_101_State_Res = transFormat_CleanDuplicated(df_101_State, 'Time_1')
df_101_Mode_Res = transFormat_CleanDuplicated(df_101_Mode, 'Time_2')
df_101_InTemp_Res = transFormat_CleanDuplicated(df_101_InTemp, 'Time_3')
df_101_SetTemp_Res = transFormat_CleanDuplicated(df_101_SetTemp, 'Time_4')
df_101_OutHum_Res = transFormat_CleanDuplicated(df_101_OutHum, 'Time_5')
df_101_OutTemp_Res = transFormat_CleanDuplicated(df_101_OutTemp, 'Time_6')


# 同步数据
## 以Mode作为主数据，再对齐其他数据


In [8]:
df_Task2 = df_101_Mode_Res
df_Task2['RoomTemp_0101'] = ''
df_Task2['SetTemp_0101'] = ''
df_Task2['OnOffState_0101'] = ''
df_Task2['OUTAIRHUMD'] = ''
df_Task2['OUTAIRTEMP'] = ''
df_101_Mode_Res.head()

,OperationModeState_0101,RoomTemp_0101,SetTemp_0101,OnOffState_0101,OUTAIRHUMD,OUTAIRTEMP
Time_2,,,,,,
2017-12-08 15:35:49,2.0,,,,,
2017-12-08 15:45:48,2.0,,,,,
2017-12-08 15:55:48,2.0,,,,,
2017-12-08 16:05:48,2.0,,,,,
2017-12-08 16:15:48,2.0,,,,,


# 重新设置索引并提取时间数据
-**提取为数组形式，加快数据处理速度**

In [9]:
df_Task2 = df_Task2.reset_index()
df_101_State_Res = df_101_State_Res.reset_index()
df_101_OutHum_Res = df_101_OutHum_Res.reset_index()
df_101_OutTemp_Res = df_101_OutTemp_Res.reset_index()
df_101_InTemp_Res = df_101_InTemp_Res.reset_index()
df_101_SetTemp_Res = df_101_SetTemp_Res.reset_index()

time_2 = df_Task2['Time_2'].tolist()
time_1 = df_101_State_Res['Time_1'].tolist()
time_5 = df_101_OutHum_Res['Time_5'].tolist()
time_6 = df_101_OutTemp_Res['Time_6'].tolist()
time_3 = df_101_InTemp_Res['Time_3'].tolist()
time_4 = df_101_SetTemp_Res['Time_4'].tolist()

Main_OnOff = df_Task2['OnOffState_0101'].tolist()
Main_OutHum = df_Task2['OUTAIRHUMD'].tolist()
Main_OutTemp = df_Task2['OUTAIRTEMP'].tolist()
Main_InTemp = df_Task2['RoomTemp_0101'].tolist()
Main_SetTemp = df_Task2['SetTemp_0101'].tolist()

time_1_OnOff = df_101_State_Res['OnOffState_0101'].tolist()
time_3_InTemp = df_101_InTemp_Res['RoomTemp_0101'].tolist()
time_4_SetTemp = df_101_SetTemp_Res['SetTemp_0101'].tolist()
time_5_OutHum = df_101_OutHum_Res['OUTAIRHUMD'].tolist()
time_6_OutTemp = df_101_OutTemp_Res['OUTAIRTEMP'].tolist()

# 转化时间为时间戳

In [10]:
for i in range(len(time_1)):
    time_1[i] = time_1[i].timestamp()
for i in range(len(time_2)):
    time_2[i] = time_2[i].timestamp()
for i in range(len(time_3)):
    time_3[i] = time_3[i].timestamp()
for i in range(len(time_4)):
    time_4[i] = time_4[i].timestamp()
for i in range(len(time_5)):
    time_5[i] = time_5[i].timestamp()
for i in range(len(time_6)):
    time_6[i] = time_6[i].timestamp()

# 对齐数据

In [11]:
# Onoff开关对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_1)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_1[j]) < 300:
            Main_OnOff[i] = time_1_OnOff[j]
            time_1_OnOff.pop(j)
            time_1.pop(j)
            break

In [12]:
#InTemp室温对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_3)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_3[j]) < 300:
            Main_InTemp[i] = time_3_InTemp[j]
            time_3_InTemp.pop(j)
            time_3.pop(j)
            break 

In [13]:
# SetTemp设温对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_4)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_4[j]) < 300:
            Main_SetTemp[i] = time_4_SetTemp[j]
            time_4_SetTemp.pop(j)
            time_4.pop(j)
            break
        

In [14]:
# OutHum室外湿度对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_5)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_5[j]) < 1800:
            Main_OutHum[i] = time_5_OutHum[j]
            break

In [15]:
# OutTemp室外温度对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_6)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_6[j]) < 1800:
            Main_OutTemp[i] = time_6_OutTemp[j]
            break

In [16]:
df_Task2['RoomTemp_0101'] = Main_InTemp
df_Task2['SetTemp_0101'] = Main_SetTemp
df_Task2['OUTAIRHUMD'] = Main_OutHum
df_Task2['OUTAIRTEMP'] = Main_OutTemp
df_Task2['OnOffState_0101'] = Main_OnOff

In [18]:
df_Task2.head()

,Time_2,OperationModeState_0101,RoomTemp_0101,SetTemp_0101,OnOffState_0101,OUTAIRHUMD,OUTAIRTEMP
0,2017-12-08 15:35:49,2.0,23.5,26,1,62.18,15.3
1,2017-12-08 15:45:48,2.0,23.5,26,1,62.18,15.1
2,2017-12-08 15:55:48,2.0,24,26,1,62.18,15.1
3,2017-12-08 16:05:48,2.0,24,26,1,62.895,15.1
4,2017-12-08 16:15:48,2.0,24,26,1,62.895,15.1


In [19]:
df_Task2.to_csv('./Datasets/101_processed.csv',index = False)

In [16]:
df_Task2

,Time_2,OperationModeState_0103,RoomTemp_0103,SetTemp_0103,OnOffState_0103,OUTAIRHUMD,OUTAIRTEMP
0,2017-12-08 15:30:52,2.0,24,24,1,62.18,15.3
1,2017-12-08 15:40:52,2.0,24,24,1,62.18,15.3
2,2017-12-08 15:50:52,2.0,24,24,1,62.18,15.1
3,2017-12-08 16:00:52,2.0,24.5,24,1,62.895,15.1
4,2017-12-08 16:10:52,2.0,24.5,26,1,62.895,15.1
5,2017-12-08 16:20:52,2.0,24.5,26,1,62.895,15.1
6,2017-12-08 16:30:52,2.0,24.5,26,0,62.895,15.1
7,2017-12-08 16:40:52,2.0,25.5,26,0,63.935,15.1
8,2017-12-08 16:50:52,2.0,25.5,26,0,63.935,15.1
9,2017-12-08 17:00:52,2.0,25,26,0,63.935,
